In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

from tqdm import tqdm


#sns.set(color_codes=True)

In [ ]:
# Make proper file names for Windows
# pip install regex

import re

def windows_filename(s):
    s = str(s).strip().replace(' ', '_')
    return re.sub(r'(?u)[^-\w.]', '', s)

In [ ]:
def plot_graph(plt,start_time, variable):
    plt.savefig(windows_filename(start_time)+'_' + variable+ '.pdf')    

In [ ]:
def ftoc(f):
    return (f - 32) * 5.0/9.0

def mphtoms(mph):
    return mph * 0.44704

In [ ]:
from pathlib import Path

basePathPatrick = Path("C:/Users/pkastner/Google Drive/Patrick/OutdoorComfort-Biking/Measurements/")
basePathESL = Path("C:/Users/pkastner/Google Drive/Patrick/OutdoorComfort-Biking/Measurements/Remy/_Stations/")
basePathRemy = Path("C:/Users/remym/Dropbox/Cornell 2019-2020/ESL Independent Study/_Stations")
basePathRemy = Path("C:/Users/remym/Dropbox/Cornell 2019-2020/ESL Independent Study/_StationDataPython")
basePathBenj = Path("C:/Users/pkastner/Google Drive/Patrick/OutdoorComfort-Biking/Measurements/Benjamin_StationFile")

loadPath = basePathPatrick

In [ ]:
from hoboreader import HoboReader
S1=HoboReader(loadPath / "20191101_11-04_DL1.csv") 
S2=HoboReader(loadPath / "20191101-10-50_DL2.csv")
S3=HoboReader(loadPath / "20191115-11-08_DL3.csv")
S4=HoboReader(loadPath / "20191101-12-58_DL4.csv")


df1=S1.get_dataframe()
df2=S2.get_dataframe()
df3=S3.get_dataframe()
df4=S4.get_dataframe()

# Variable to plot
variable1 = "Wind Speed"
variable2 = "Wind Direction"

#df1[variable].hist()
sns.distplot(df1[variable1], kde=False, fit=stats.gamma);
sns.distplot(df2[variable1], kde=False, fit=stats.gamma);
sns.distplot(df3[variable1], kde=False, fit=stats.gamma);
sns.distplot(df4[variable1], kde=False, fit=stats.gamma);

#sns.jointplot(x=variable1, y=variable2, data=df1);

In [ ]:
from windrose import WindroseAxes
from windrose import WindAxes
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import numpy as np

def plotWR(ws, wd):
    ax = WindroseAxes.from_ax()
    ax.contourf(wd, ws, bins=np.arange(0, 6, 1),nsector= 36, cmap=cm.hot)
    ax.contour(wd, ws, bins=np.arange(0, 6, 1),nsector= 36, colors='black', lw=1)
    ax.set_legend()
    
def printPDF(ws):
    ax = WindAxes.from_ax()
    bins = np.arange(0, 6 + 1, 0.5)
    bins = bins[1:]
    ax, params = ax.pdf(ws, bins=bins)

In [ ]:


list = [df1, df2, df3, df4]

# Variable to plot
variable1 = "Wind Speed"
variable2 = "Wind Direction"

ws = np.array(df1[variable1]).flatten()
wd = np.array(df1[variable2]).flatten()

for i in tqdm(list):
    ws = np.array(i[variable1]).flatten()
    wd = np.array(i[variable2]).flatten()
    plotWR(ws, wd)
    printPDF(ws)
    
     
    


In [ ]:
from hoboreader import HoboReader
from pandas import read_csv
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from pathlib import Path

      
    
#DL1	Stone Garden - Tjaden Hall
#DL2	PSB Parking Lot
#DL3	Olin Library
#DL4	Milstein Hall Roof   
  
    
S1=HoboReader(loadPath / "20191101_11-04_DL1.csv") 
S2=HoboReader(loadPath / "20191101-10-50_DL2.csv")
S3=HoboReader(loadPath / "20191115-11-08_DL3.csv")
S4=HoboReader(loadPath / "20191101-12-58_DL4.csv")


df1=S1.get_dataframe()
df2=S2.get_dataframe()
df3=S3.get_dataframe()
df4=S4.get_dataframe()

# Select time

start_time = '2019-10-17 06:00:00-04:00'
end_time = '2019-10-17 19:00:00-04:00'

df1_ = df1.loc[start_time:end_time]
df2_ = df2.loc[start_time:end_time]
df3_ = df3.loc[start_time:end_time]
df4_ = df4.loc[start_time:end_time]

#df1.index

# Print Headers
#print(df1.columns.values)


# Resize figure
custom_figsize=(15,5)

# Change lineweights
lineweights = 2

# Variable to plot
variable = 'Solar Radiation'

# Plot two dataframes into one plot
ax = df1_.plot(y=[variable],lw = lineweights, figsize=custom_figsize,color='blue')
df3_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='yellow',ax=ax)
df2_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='red',ax=ax)
df4_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='magenta',ax=ax)

# Subset Time Series By Dates Python Using Pandas
#https://www.earthdatascience.org/courses/earth-analytics-python/use-time-series-data-in-python/subset-time-series-data-python/


# Legend
ax.legend([
"DL1 - Stone Garden - Tjaden Hall", 
"DL2 - PSB Parking Lot",
"DL3 - Olin Library",
"DL4 - Milstein Hall Roof "  
],loc='upper left', frameon=False)


# Fix xticks

years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
days = mdates.DayLocator()  # every minute
hours = mdates.HourLocator()  # every minute
minutes = mdates.MinuteLocator()  # every minute

days_fmt = mdates.DateFormatter('%d')
hours_fmt = mdates.DateFormatter('%H:00')

# format the ticks
ax.xaxis.set_major_locator(hours)
ax.xaxis.set_major_formatter(hours_fmt)
ax.xaxis.set_minor_locator(hours)

#Labels
plt.ylabel('Global Radiation $[W/m^2]$')
plt.xlabel("Time")
     
# Title
ax.set_title(variable)

# Save PDF
plot_graph(plt, start_time, variable)

In [ ]:
from hoboreader import HoboReader
from pandas import read_csv
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from pathlib import Path
     
    
#DL1	Stone Garden - Tjaden Hall
#DL2	PSB Parking Lot
#DL3	Olin Library
#DL4	Milstein Hall Roof   
  
    
S1=HoboReader(loadPath / "20191101_11-04_DL1.csv") 
S2=HoboReader(loadPath / "20191101-10-50_DL2.csv")
S3=HoboReader(loadPath / "20191115-11-08_DL3.csv")
S4=HoboReader(loadPath / "20191101-12-58_DL4.csv")


df1=S1.get_dataframe()
df2=S2.get_dataframe()
df3=S3.get_dataframe()
df4=S4.get_dataframe()

# Select time

start_time = '2019-10-17 06:00:00-04:00'
end_time = '2019-10-17 19:00:00-04:00'

df1_ = df1.loc[start_time:end_time]
df2_ = df2.loc[start_time:end_time]
df3_ = df3.loc[start_time:end_time]
df4_ = df4.loc[start_time:end_time]

#df1.index

# Print Headers
#print(df1.columns.values)


# Resize figure
custom_figsize=(15,5)

# Change lineweights
lineweights = 2

# Variable to plot
variable = "Wind Speed"

# Plot two dataframes into one plot
ax = df1_.plot(y=[variable],lw = lineweights, figsize=custom_figsize,color='blue')
df3_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='yellow',ax=ax)
df2_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='red',ax=ax)
df4_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='magenta',ax=ax)

# Subset Time Series By Dates Python Using Pandas
#https://www.earthdatascience.org/courses/earth-analytics-python/use-time-series-data-in-python/subset-time-series-data-python/


# Legend
ax.legend([
"DL1 - Stone Garden - Tjaden Hall", 
"DL2 - PSB Parking Lot",
"DL3 - Olin Library",
"DL4 - Milstein Hall Roof "  
],loc='upper left', frameon=False)


# Fix xticks

years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
days = mdates.DayLocator()  # every minute
hours = mdates.HourLocator()  # every minute
minutes = mdates.MinuteLocator()  # every minute

days_fmt = mdates.DateFormatter('%d')
hours_fmt = mdates.DateFormatter('%H:00')

# format the ticks
ax.xaxis.set_major_locator(hours)
ax.xaxis.set_major_formatter(hours_fmt)
ax.xaxis.set_minor_locator(hours)

#Labels
plt.ylabel('Wind velocity $[m/s]$')
plt.xlabel("Time")
     
# Title
ax.set_title(variable)

# Save PDF
plot_graph(plt, start_time, variable)

In [ ]:
#DL1	Stone Garden - Tjaden Hall
#DL2	PSB Parking Lot
#DL3	Olin Library
#DL4	Milstein Hall Roof   


#"#","Date Time, GMT-04:00",
#"Temp, °F (LGR S/N: 20702490, SEN S/N: 20496098, LBL: MRT)",
#"Temp, °F (LGR S/N: 20702490, SEN S/N: 20698454, LBL: Drybulb)",
#"RH, % (LGR S/N: 20702490, SEN S/N: 20698454)",
#"Solar Radiation, W/m² (LGR S/N: 20702490, SEN S/N: 20702165, LBL: Rad)",
#"Wind Speed, mph (LGR S/N: 20702490, SEN S/N: 20706788, LBL: Wind)",
#"Gust Speed, mph (LGR S/N: 20702490, SEN S/N: 20706788)",
#"Wind Direction, ø (LGR S/N: 20702490, SEN S/N: 20706788)",
#"DewPt, °F (LGR S/N: 20702490, SEN S/N: 20698454)"
    

S4=HoboReader(loadPath / "20191101-12-58_DL4.csv")


df4=S4.get_dataframe()

# Select time

start_time = '2019-09-01 06:00:00-04:00'
end_time = '2019-12-15 19:00:00-04:00'


df4_ = df4.loc[start_time:end_time]

# Resize figure
custom_figsize=(25,5)

# Change lineweights
lineweights = 2


# Variable to plot
variable = "Temp"
variable2 = "Temp"


test= df4_[variable].subtract(df4_[variable2], fill_value=0)


print(test)


# Plot two dataframes into one plot
#ax = plt.plot(diff,lw = lineweights, figsize=custom_figsize,color='blue')
#
## Subset Time Series By Dates Python Using Pandas
##https://www.earthdatascience.org/courses/earth-analytics-python/use-time-series-data-in-python/subset-time-series-data-python/
#
## Legend
#ax.legend([
#"DL1 - Stone Garden - Tjaden Hall", 
#"DL2 - PSB Parking Lot",
#"DL3 - Olin Library",
#"DL4 - Milstein Hall Roof "  
#],loc='upper left', frameon=False)
#
## Fix xticks
#
#years = mdates.YearLocator()   # every year
#months = mdates.MonthLocator()  # every month
#days = mdates.DayLocator()  # every minute
#hours = mdates.HourLocator()  # every minute
#minutes = mdates.MinuteLocator()  # every minute
#
#days_fmt = mdates.DateFormatter('%d')
#hours_fmt = mdates.DateFormatter('%H:00')
#
## format the ticks
#ax.xaxis.set_major_locator(hours)
#ax.xaxis.set_major_formatter(hours_fmt)
#ax.xaxis.set_minor_locator(hours)
#
##Labels
#plt.ylabel('Wind velocity $[m/s]$')
#plt.xlabel("Time")
#     
## Title
#ax.set_title(variable)

In [ ]:
from hoboreader import HoboReader
from pandas import read_csv
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from pathlib import Path

    
#DL1	Stone Garden - Tjaden Hall
#DL2	PSB Parking Lot
#DL3	Olin Library
#DL4	Milstein Hall Roof   
  
    
S1=HoboReader(loadPath / "20191101_11-04_DL1.csv") 
S2=HoboReader(loadPath / "20191101-10-50_DL2.csv")
S3=HoboReader(loadPath / "20191115-11-08_DL3.csv")
S4=HoboReader(loadPath / "20191101-12-58_DL4.csv")


df1=S1.get_dataframe()
df2=S2.get_dataframe()
df3=S3.get_dataframe()
df4=S4.get_dataframe()

# Select time

start_time = '2019-10-17 06:00:00-04:00'
end_time = '2019-10-17 19:00:00-04:00'

df1_ = df1.loc[start_time:end_time]
df2_ = df2.loc[start_time:end_time]
df3_ = df3.loc[start_time:end_time]
df4_ = df4.loc[start_time:end_time]

#df1.index

# Print Headers
#print(df1.columns.values)


# Resize figure
custom_figsize=(15,5)

# Change lineweights
lineweights = 2

# Variable to plot
variable = "Wind Direction"

# Plot two dataframes into one plot
ax = df1_.plot(y=[variable],lw = lineweights, figsize=custom_figsize,color='blue')
df3_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='yellow',ax=ax)
df2_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='red',ax=ax)
df4_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='magenta',ax=ax)

# Subset Time Series By Dates Python Using Pandas
#https://www.earthdatascience.org/courses/earth-analytics-python/use-time-series-data-in-python/subset-time-series-data-python/


# Legend
ax.legend([
"DL1 - Stone Garden - Tjaden Hall", 
"DL2 - PSB Parking Lot",
"DL3 - Olin Library",
"DL4 - Milstein Hall Roof "  
],loc='upper left', frameon=False)


# Fix xticks

years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
days = mdates.DayLocator()  # every minute
hours = mdates.HourLocator()  # every minute
minutes = mdates.MinuteLocator()  # every minute

days_fmt = mdates.DateFormatter('%d')
hours_fmt = mdates.DateFormatter('%H:00')

# format the ticks
ax.xaxis.set_major_locator(hours)
ax.xaxis.set_major_formatter(hours_fmt)
ax.xaxis.set_minor_locator(hours)

#Labels
plt.ylabel('Wind direction $[°]$')
plt.xlabel("Time")
     
# Title
ax.set_title(variable)

# Save PDF
plot_graph(plt, start_time, variable)

In [ ]:
# conda install -c https://conda.anaconda.org/ioos geopandas
# conda install -c conda-forge shapely
# conda install -c conda-forge descartes

import geopandas as gpd
from shapely.geometry import Point, Polygon
import descartes

#final_crs = {'init': 'epsg:28992'}

buildings = gpd.read_file('Ithaca/map/buildings-polygon.shp')#.to_crs(final_crs)
roads = gpd.read_file('Ithaca/map/roads-line.shp')#.to_crs(final_crs)
landcover = gpd.read_file('Ithaca/map/landcover-polygon.shp')#.to_crs(final_crs)
water = gpd.read_file('Ithaca/map/water_areas-polygon.shp')#.to_crs(final_crs)
waterlines = gpd.read_file('Ithaca/map/water_lines-line.shp')#.to_crs(final_crs)

fig,ax = plt.subplots(figsize= (10,10))

roads.plot(ax=ax, color= "grey")
landcover.plot(ax=ax, color= "green", alpha=0.03)
water.plot(ax=ax, color= "blue", alpha=0.1)
waterlines.plot(ax=ax, color= "blue", alpha=0.1)
buildings.plot(ax=ax, color= "black")

minx, miny, maxx, maxy = -76.4900, 42.443484, -76.475089, 42.4555
ax.set_xlim(minx, maxx)
ax.set_ylim(miny, maxy)

DL1= Point((-76.485089,42.450960))
DL2= Point((-76.481194,42.450536))
DL3= Point((-76.484268,42.448039))
DL4= Point((-76.483743,42.451245))

gdf_stations = gpd.GeoSeries([DL1,DL2,DL3,DL4])
gdf_stations.plot(ax=ax, markersize= 50, color= "red", marker = "o")

# Annotate with names

stations = [
"DL1", 
"DL2",
"DL3",
"DL4"  
]
xx = 0.0003
yy = 0.0001

for i, geo in gdf_stations.centroid.iteritems():
    ax.annotate(s=stations[i], xy=[geo.x+xx, geo.y-yy], color="red" )


# Save PDF
plt.savefig('Map.pdf')

In [ ]:
from hoboreader import HoboReader
from pandas import read_csv
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from pathlib import Path


    
#DL1	Stone Garden - Tjaden Hall
#DL2	PSB Parking Lot
#DL3	Olin Library
#DL4	Milstein Hall Roof   
  
    
S1=HoboReader(basePathPatrick / "20191101_11-04_DL1.csv") 
S2=HoboReader(basePathPatrick / "20191101-10-50_DL2.csv")
S3=HoboReader(basePathPatrick / "20191115-11-08_DL3.csv")
S4=HoboReader(basePathPatrick / "20191101-12-58_DL4.csv")


df1=S1.get_dataframe()
df2=S2.get_dataframe()
df3=S3.get_dataframe()
df4=S4.get_dataframe()

# Select time

start_time = '2019-10-31 00:00:00-04:00'
end_time = '2019-11-01 23:59:00-04:00'

df1_ = df1.loc[start_time:end_time]
df2_ = df2.loc[start_time:end_time]
df3_ = df3.loc[start_time:end_time]
df4_ = df4.loc[start_time:end_time]

#df1.index

# Print Headers
#print(df1.columns.values)


# Resize figure
custom_figsize=(40,15)

# Change lineweights
lineweights = 1.5

# Variable to plot
variable = "Wind Direction"

# Plot two dataframes into one plot
ax = df1_.plot(y=[variable],lw = lineweights, figsize=custom_figsize,color='red')
df3_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='blue',ax=ax)
df2_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='green',ax=ax)
df4_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='aqua',ax=ax)

# Subset Time Series By Dates Python Using Pandas
#https://www.earthdatascience.org/courses/earth-analytics-python/use-time-series-data-in-python/subset-time-series-data-python/


# Legend
ax.legend([
"DL1 - Stone Garden - Tjaden Hall", 
"DL2 - PSB Parking Lot",
"DL3 - Olin Library",
"DL4 - Milstein Hall Roof "  
],loc='upper left', frameon=False)


# Fix xticks

years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
days = mdates.DayLocator()  # every minute
hours = mdates.HourLocator()  # every minute
minutes = mdates.MinuteLocator()  # every minute

days_fmt = mdates.DateFormatter('%d')
hours_fmt = mdates.DateFormatter('%H:00')

# format the ticks
ax.xaxis.set_major_locator(hours)
ax.xaxis.set_major_formatter(hours_fmt)
ax.xaxis.set_minor_locator(hours)

#Labels
plt.ylabel('Wind direction $[°]$', fontname="Consolas", fontsize=20)
plt.xlabel("Time", fontname="Consolas", fontsize=20)
     
# Title
ax.set_title(variable, fontname="Consolas", fontsize=28)

# Save PDF
plot_graph(plt, start_time, variable)

In [ ]:
from hoboreader import HoboReader
from pandas import read_csv
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from pathlib import Path


      
    
#DL1	Stone Garden - Tjaden Hall
#DL2	PSB Parking Lot
#DL3	Olin Library
#DL4	Milstein Hall Roof   
  
    
S1=HoboReader(basePathPatrick/ "20191101_11-04_DL1.csv") 
S2=HoboReader(basePathPatrick/ "20191101-10-50_DL2.csv")
S3=HoboReader(basePathPatrick/ "20191115-11-08_DL3.csv")
S4=HoboReader(basePathPatrick/ "20191101-12-58_DL4.csv")


df1=S1.get_dataframe()
df2=S2.get_dataframe()
df3=S3.get_dataframe()
df4=S4.get_dataframe()

# Select time

start_time = '2019-10-31 00:00:00-04:00'
end_time = '2019-11-01 23:59:00-04:00'

df1_ = df1.loc[start_time:end_time]
df2_ = df2.loc[start_time:end_time]
df3_ = df3.loc[start_time:end_time]
df4_ = df4.loc[start_time:end_time]

#df1.index

# Print Headers
#print(df1.columns.values)


# Resize figure
custom_figsize=(40,15)

# Change lineweights
lineweights = 1.5

# Variable to plot
variable = "Wind Speed"

# Plot two dataframes into one plot
ax = df1_.plot(y=[variable],lw = lineweights, figsize=custom_figsize,color='red')
df3_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='blue',ax=ax)
df2_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='green',ax=ax)
df4_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='aqua',ax=ax)

# Subset Time Series By Dates Python Using Pandas
#https://www.earthdatascience.org/courses/earth-analytics-python/use-time-series-data-in-python/subset-time-series-data-python/


# Legend
ax.legend([
"DL1 - Stone Garden - Tjaden Hall", 
"DL2 - PSB Parking Lot",
"DL3 - Olin Library",
"DL4 - Milstein Hall Roof "  
],loc='upper left', frameon=False)


# Fix xticks

years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
days = mdates.DayLocator()  # every minute
hours = mdates.HourLocator()  # every minute
minutes = mdates.MinuteLocator()  # every minute

days_fmt = mdates.DateFormatter('%d')
hours_fmt = mdates.DateFormatter('%H:00')

# format the ticks
ax.xaxis.set_major_locator(hours)
ax.xaxis.set_major_formatter(hours_fmt)
ax.xaxis.set_minor_locator(hours)

#Labels
plt.ylabel('Wind Speed $[MPH]$', fontname="Consolas", fontsize=20)
plt.xlabel("Time", fontname="Consolas", fontsize=20)
     
# Title
ax.set_title(variable, fontname="Consolas", fontsize=28)

# Save PDF
plot_graph(plt, start_time, variable)

In [ ]:
from hoboreader import HoboReader
from pandas import read_csv
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from pathlib import Path


    
#DL1	Stone Garden - Tjaden Hall
#DL2	PSB Parking Lot
#DL3	Olin Library
#DL4	Milstein Hall Roof   
  
    
S1=HoboReader(basePathPatrick / "20191101_11-04_DL1.csv") 
S2=HoboReader(basePathPatrick / "20191101-10-50_DL2.csv")
S3=HoboReader(basePathPatrick / "20191115-11-08_DL3.csv")
S4=HoboReader(basePathPatrick / "20191101-12-58_DL4.csv")


df1=S1.get_dataframe()
df2=S2.get_dataframe()
df3=S3.get_dataframe()
df4=S4.get_dataframe()

# Select time

start_time = '2019-10-31 00:00:00-04:00'
end_time = '2019-11-01 23:59:00-04:00'

df1_ = df1.loc[start_time:end_time]
df2_ = df2.loc[start_time:end_time]
df3_ = df3.loc[start_time:end_time]
df4_ = df4.loc[start_time:end_time]

#df1.index

# Print Headers
#print(df1.columns.values)


# Resize figure
custom_figsize=(40,15)

# Change lineweights
lineweights = 2

# Variable to plot
variable = 'Solar Radiation'

# Plot two dataframes into one plot
ax = df1_.plot(y=[variable],lw = lineweights, figsize=custom_figsize,color='red')
df3_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='blue',ax=ax)
df2_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='green',ax=ax)
df4_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='aqua',ax=ax)

# Subset Time Series By Dates Python Using Pandas
#https://www.earthdatascience.org/courses/earth-analytics-python/use-time-series-data-in-python/subset-time-series-data-python/


# Legend
ax.legend([
"DL1 - Stone Garden - Tjaden Hall", 
"DL2 - PSB Parking Lot",
"DL3 - Olin Library",
"DL4 - Milstein Hall Roof "  
],loc='upper left', frameon=False)


# Fix xticks

years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
days = mdates.DayLocator()  # every minute
hours = mdates.HourLocator()  # every minute
minutes = mdates.MinuteLocator()  # every minute

days_fmt = mdates.DateFormatter('%d')
hours_fmt = mdates.DateFormatter('%H:00')

# format the ticks
ax.xaxis.set_major_locator(hours)
ax.xaxis.set_major_formatter(hours_fmt)
ax.xaxis.set_minor_locator(hours)

#Labels
plt.ylabel('Global Radiation $[W/m^2]$', fontname="Consolas", fontsize=20)
plt.xlabel("Time", fontname="Consolas", fontsize=20)
     
# Title
ax.set_title(variable, fontname="Consolas", fontsize=28)

# Save PDF
plot_graph(plt, start_time, variable)

In [ ]:
from hoboreader import HoboReader
from pandas import read_csv
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from pathlib import Path


    
#DL1	Stone Garden - Tjaden Hall
#DL2	PSB Parking Lot
#DL3	Olin Library
#DL4	Milstein Hall Roof   
  
    
S1=HoboReader(basePathPatrick / "20191101_11-04_DL1.csv") 
S2=HoboReader(basePathPatrick / "20191101-10-50_DL2.csv")
S3=HoboReader(basePathPatrick / "20191115-11-08_DL3.csv")
S4=HoboReader(basePathPatrick / "20191101-12-58_DL4.csv")


df1=S1.get_dataframe()
df2=S2.get_dataframe()
df3=S3.get_dataframe()
df4=S4.get_dataframe()

# Select time

start_time = '2019-10-31 00:00:00-04:00'
end_time = '2019-11-01 23:59:00-04:00'

df1_ = df1.loc[start_time:end_time]
df2_ = df2.loc[start_time:end_time]
df3_ = df3.loc[start_time:end_time]
df4_ = df4.loc[start_time:end_time]

#df1.index

# Print Headers
#print(df1.columns.values)


# Resize figure
custom_figsize=(40,15)

# Change lineweights
lineweights = 1.5

# Variable to plot
variable = "Temp"

# Translate to °C

df1_[variable] = df1_[variable].apply(lambda x: f2c(x), axis=1)
df2_[variable] = df2_[variable].apply(lambda x: f2c(x), axis=1)
df3_[variable] = df3_[variable].apply(lambda x: f2c(x), axis=1)
df4_[variable] = df4_[variable].apply(lambda x: f2c(x), axis=1)


# Plot two dataframes into one plot
ax = df1_.plot(y=[variable],lw = lineweights, figsize=custom_figsize,color='red')
df3_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='blue',ax=ax)
df2_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='green',ax=ax)
df4_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='aqua',ax=ax)

# Subset Time Series By Dates Python Using Pandas
#https://www.earthdatascience.org/courses/earth-analytics-python/use-time-series-data-in-python/subset-time-series-data-python/


# Legend
ax.legend([
"DL1 - Stone Garden - Tjaden Hall", 
"DL2 - PSB Parking Lot",
"DL3 - Olin Library",
"DL4 - Milstein Hall Roof "  
],loc='upper left', frameon=False)


# Fix xticks

years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
days = mdates.DayLocator()  # every minute
hours = mdates.HourLocator()  # every minute
minutes = mdates.MinuteLocator()  # every minute

days_fmt = mdates.DateFormatter('%d')
hours_fmt = mdates.DateFormatter('%H:00')

# format the ticks
ax.xaxis.set_major_locator(hours)
ax.xaxis.set_major_formatter(hours_fmt)
ax.xaxis.set_minor_locator(hours)

#Labels
plt.ylabel('Temp $[°C]$', fontname="Consolas", fontsize=20)
plt.xlabel("Time", fontname="Consolas", fontsize=20)
     
# Title
ax.set_title(variable, fontname="Consolas", fontsize=28)

# Save PDF
plot_graph(plt, start_time, variable)

In [ ]:
# conda install -c https://conda.anaconda.org/ioos geopandas
# conda install -c conda-forge shapely
# conda install -c conda-forge descartes

import geopandas as gpd
from shapely.geometry import Point, Polygon
import descartes

#final_crs = {'init': 'epsg:28992'}

buildings = gpd.read_file('Ithaca/map/buildings-polygon.shp')#.to_crs(final_crs)
roads = gpd.read_file('Ithaca/map/roads-line.shp')#.to_crs(final_crs)
landcover = gpd.read_file('Ithaca/map/landcover-polygon.shp')#.to_crs(final_crs)
water = gpd.read_file('Ithaca/map/water_areas-polygon.shp')#.to_crs(final_crs)
waterlines = gpd.read_file('Ithaca/map/water_lines-line.shp')#.to_crs(final_crs)

fig,ax = plt.subplots(figsize= (10,10))

roads.plot(ax=ax, color= "grey")
landcover.plot(ax=ax, color= "green", alpha=0.03)
water.plot(ax=ax, color= "blue", alpha=0.1)
waterlines.plot(ax=ax, color= "blue", alpha=0.1)
buildings.plot(ax=ax, color= "black")

minx, miny, maxx, maxy = -76.4900, 42.443484, -76.475089, 42.4555
ax.set_xlim(minx, maxx)
ax.set_ylim(miny, maxy)

DL1= Point((-76.485089,42.450960))
DL2= Point((-76.481194,42.450536))
DL3= Point((-76.484268,42.448039))
DL4= Point((-76.483743,42.451245))

gdf_stations = gpd.GeoSeries([DL1,DL2,DL3,DL4])
gdf_stations.plot(ax=ax, markersize= 50, color= "red", marker = "o")

# Annotate with names

stations = [
"DL1", 
"DL2",
"DL3",
"DL4"  
]
xx = 0.0003
yy = 0.0001

for i, geo in gdf_stations.centroid.iteritems():
    ax.annotate(s=stations[i], xy=[geo.x+xx, geo.y-yy], color="red" )


# Save PDF
plt.savefig('Map.pdf')

In [2]:
# Merge all recorded data



from hoboreader import HoboReader
from pandas import read_csv
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from pathlib import Path
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd
%matplotlib inline


pd.set_option('mode.chained_assignment',None)

# Set up figure size and DPI for screen demo
plt.rcParams['figure.figsize'] = (6,4)
plt.rcParams['figure.dpi'] = 150


#loadPath = Path(r"C:\Users\remym\Dropbox\ESL\CSV Merged Data")
#loadPath = Path(r"C:\Users\pkastner\Google Drive\Patrick\OutdoorComfort-Biking\Measurements")

#S1=HoboReader(loadPath / "DL1_Stone_Garden_Tjaden_Hall.csv") 
#S2=HoboReader(loadPath / "DL2_PSB_Parking_Lot.csv")
#S3=HoboReader(loadPath / "DL3_Olin_Library.csv")
#S4=HoboReader(loadPath / "DL4_Milstein_Hall_Roof.csv")
#S5=HoboReader(LoadPath / "DL5_Game_Farm_Road_20192020v4.csv")

    
#DL1	Stone Garden - Tjaden Hall
#DL2	PSB Parking Lot
#DL3	Olin Library
#DL4	Milstein Hall Roof   
#DL5	Game Farm Road   
  
    
S1=HoboReader("DL1_Stone_Garden_Tjaden_Hall.csv") 
S2=HoboReader("DL2_PSB_Parking_Lot.csv")
S3=HoboReader("DL3_Olin_Library.csv")
S4=HoboReader("DL4_Milstein_Hall_Roof.csv")
S5=HoboReader("DL5_Game_Farm_Road_20192020v4.csv")



df1=S1.get_dataframe()
df2=S2.get_dataframe()
df3=S3.get_dataframe()
df4=S4.get_dataframe()
df5=S5.get_dataframe()

#Shift the measurements back and forth so they are all in alignment with the GFR station
#if start_time == "2019-03-03", end_time == "2019-11-03 03:00:00" : 
   #df5=df5.shift(periods=(-1), fill_value=0 )
#if start_time == "2019-11-03 04:00:00" and end_time == "2020-03-08 03:00:00" : 
   #df5=df5.shift(periods=(0), fill_value=0 )

#DF5 (GAME FARM ROAD) HAS TO BE SHIFTED 1HR BACK BECAUSE IT NATIVELY RECORDS DST, OUR STATIONS DO NOT. 
#THE OTHER STATIONS ARE NOT SHIFTED (HENCE 0 VALUE FOR PERIODS), JUST THERE FOR SANITY

df5=df5.shift(periods=(-1), fill_value=0 )   
df4=df4.shift(periods=(0), fill_value=0 )
df3=df3.shift(periods=(0), fill_value=0 )
df2=df2.shift(periods=(0), fill_value=0 )
df1=df1.shift(periods=(0), fill_value=0 )

#Resample subhourly data to 1 hourly graph (OUR STATIONS RECORD IN 10MIN INTERVALS, GFR IN 1HR, RESAMPLE FOR EASY COMPARISON. CAN TURN OFF)
df1=df1.resample('H').mean()
df2=df2.resample('H').mean()
df3=df3.resample('H').mean()
df4=df4.resample('H').mean()
df5=df5.resample('H').mean()

#CONVERT IMPERIAL TO METRIC UNITS

def ftoc(f):
    return (f - 32) * 5.0/9.0

def mphtoms(mph):
    return mph * 0.44704

def f(variable_id,d, m,y):
            
    start_time = str(y)+'-'+str(m)+'-'+str(d)+' 07:00:00-00:00'
    end_time =   str(y)+'-'+str(m)+'-'+str(d)+' 23:00:00-00:00'
    
   
    df1_ = df1.loc[start_time:end_time]
    df2_ = df2.loc[start_time:end_time]
    df3_ = df3.loc[start_time:end_time]
    df4_ = df4.loc[start_time:end_time]
    df5_ = df5.loc[start_time:end_time]


    # Translate to °C

    df1_["Temp"] = df1_["Temp"].apply(lambda x: ftoc(x), axis=1)
    df2_["Temp"] = df2_["Temp"].apply(lambda x: ftoc(x), axis=1)
    df3_["Temp"] = df3_["Temp"].apply(lambda x: ftoc(x), axis=1)
    df4_["Temp"] = df4_["Temp"].apply(lambda x: ftoc(x), axis=1)
    df5_["Temp"] = df5_["Temp"].apply(lambda x: ftoc(x), axis=1)
    
    df1_["MRT"] = df1_["MRT"].apply(lambda x: ftoc(x), axis=1)
    df2_["MRT"] = df2_["MRT"].apply(lambda x: ftoc(x), axis=1)
    df3_["MRT"] = df3_["MRT"].apply(lambda x: ftoc(x), axis=1)
    df4_["MRT"] = df4_["MRT"].apply(lambda x: ftoc(x), axis=1)
    #df5_["MRT"] = df5_["MRT"].apply(lambda x: ftoc(x), axis=1)
    
    # Translate to m/s
    
    df1_["Wind Speed"] = df1_["Wind Speed"].apply(lambda x: mphtoms(x), axis=1)
    df2_["Wind Speed"] = df2_["Wind Speed"].apply(lambda x: mphtoms(x), axis=1)
    df3_["Wind Speed"] = df3_["Wind Speed"].apply(lambda x: mphtoms(x), axis=1)
    df4_["Wind Speed"] = df4_["Wind Speed"].apply(lambda x: mphtoms(x), axis=1)
    df5_["Wind Speed"] = df5_["Wind Speed"].apply(lambda x: mphtoms(x), axis=1)
    
    df1_["Gust Speed"] = df1_["Gust Speed"].apply(lambda x: mphtoms(x), axis=1)
    df2_["Gust Speed"] = df2_["Gust Speed"].apply(lambda x: mphtoms(x), axis=1)
    df3_["Gust Speed"] = df3_["Gust Speed"].apply(lambda x: mphtoms(x), axis=1)
    df4_["Gust Speed"] = df4_["Gust Speed"].apply(lambda x: mphtoms(x), axis=1)
    #df5_["Gust Speed"] = df5_["Gust Speed"].apply(lambda x: mphtoms(x), axis=1)
    
    
    #df1.index
    
    # Print Headers
    #print(df1.columns.values)
    
    
    # Resize figure
    custom_figsize=(20,10)
    
    # Change lineweights
    lineweights = 1.5
    

    

    
    variables = ["Wind Speed","MRT","Temp","RH","Solar Radiation","Wind Speed","Gust Speed","Wind Direction","DewPt" ]  
    
    
    
    variable = variables[variable_id]
    
    # Plot both temp and mrt when temp is selected
    
    if variable == "MRT" or variable == "DewPt" or variable == "Gust Speed" :
        ax = df1_.plot(y = variable, lw=1, figsize=custom_figsize,color='red')
        df2_.plot( y = variable, lw=1, figsize=custom_figsize,color='blue',ax=ax)
        df3_.plot(y = variable, lw=1, figsize=custom_figsize,color='green',ax=ax)
        df4_.plot(y = variable, lw=1, figsize=custom_figsize,color='aqua',ax=ax)  
    
    elif variable == "Temp":
        # Plot two dataframes into one plot
        ax = df1_.plot(y = variable, lw=2, figsize=custom_figsize,color='red')
        df2_.plot( y = variable,    lw=2, figsize=custom_figsize,color='blue',ax=ax)
        df3_.plot(y = variable,     lw=2, figsize=custom_figsize,color='green',ax=ax)
        df4_.plot(y = variable,     lw=2, figsize=custom_figsize,color='aqua',ax=ax)
        df5_.plot(y = variable,     lw=3.5, figsize=custom_figsize,color='black',ax=ax)
        
        ax = df1_.plot(y = "MRT", lw=1, figsize=custom_figsize,style = ['--'],color='red', ax=ax)
        df2_.plot( y = "MRT",     lw=1, figsize=custom_figsize, style = ['--'],color='blue',ax=ax)
        df3_.plot(y = "MRT",      lw=1, figsize=custom_figsize, style = ['--'],color='green',ax=ax)
        df4_.plot(y = "MRT",      lw=1, figsize=custom_figsize, style = ['--'],color='aqua',ax=ax)      
                  
    else:  
        # Plot two dataframes into one plot
        ax = df1_.plot(y = variable, lw=1, figsize=custom_figsize,color='red')
        df2_.plot( y = variable, lw=1, figsize=custom_figsize,color='blue',ax=ax)
        df3_.plot(y = variable, lw=1, figsize=custom_figsize,color='green',ax=ax)
        df4_.plot(y = variable, lw=1, figsize=custom_figsize,color='aqua',ax=ax)
        df5_.plot(y = variable, lw=2, figsize=custom_figsize,color='black',ax=ax)
    
    # Legend
    ax.legend([
    "DL1 - Stone Garden - Tjaden Hall", 
    "DL2 - PSB Parking Lot",
    "DL3 - Olin Library",
    "DL4 - Milstein Hall Roof ",
    "DL5 - Game Farm Road"
    
    ],loc='upper left', frameon=False)
    
    
    # Fix xticks
    
    years = mdates.YearLocator()   # every year
    months = mdates.MonthLocator()  # every month
    days = mdates.DayLocator()  # every minute
    hours = mdates.HourLocator()  # every minute
    minutes = mdates.MinuteLocator()  # every minute
    
    days_fmt = mdates.DateFormatter('%d')
    hours_fmt = mdates.DateFormatter('%H:00')
    
    # format the ticks
    ax.xaxis.set_major_locator(hours)
    ax.xaxis.set_major_formatter(hours_fmt)
    ax.xaxis.set_minor_locator(hours)
    
    #Labels
    plt.ylabel(variable, fontname="Consolas", fontsize=20)
    plt.xlabel("UTC Time", fontname="Consolas", fontsize=20)
         
    # Title
    ax.set_title(str(start_time.split(' ')[0]+" "+variable), fontname="Consolas", fontsize=28)
    
    # Save PDF
    #plot_graph(plt, start_time, variable)

    
# Variable to plot
#1 = "MRT °C"
#2 = "Temp °C"
#3 = "RH"
#4 = "Solar Radiation"
#5 = "Wind Speed"
#6 = "Gust Speed"
#7 = "Wind Direction"
#8 = "DewPt"
    
    
interact(f,variable_id=(1,8,1), d=(1,31,1), m=(1,12,1), y = (2019,2020,1))










interactive(children=(IntSlider(value=4, description='variable_id', max=8, min=1), IntSlider(value=16, descrip…

<function __main__.f(variable_id, d, m, y)>

In [ ]:
display(df5)

# Direct/Diffuse split


We need the extra terrestrial radiation for the diffuse part which we can get with "get_extra_radiation()".

https://pvlib-python.readthedocs.io/en/stable/generated/pvlib.irradiance.get_extra_radiation.html?highlight=dni_extra%20

From there, we can use "reindl()" to calculate the split.

https://pvlib-python.readthedocs.io/en/stable/generated/pvlib.irradiance.reindl.html?highlight=reindl


https://pvpmc.sandia.gov/modeling-steps/1-weather-design-inputs/plane-of-array-poa-irradiance/

In [4]:
from hoboreader import HoboReader
S5 = HoboReader("DL5_Game_Farm_Road_2019.csv")
df5 = S5.get_dataframe()


!pip install pvlib
import pvlib
#https://pvlib-python.readthedocs.io/en/stable/generated/pvlib.irradiance.reindl.html?highlight=reindl#pvlib.irradiance.reindl

# conda install -c pvlib pvlib
import pandas as pd

# Create date series using date_range() function
date_series = pd.date_range('01/01/2019', periods = 8760, freq ='H')
#print(date_series)

# altitude GFR Station
#altitude = 294

# lat, lon taken from GFR
lon = 42.449 # north-positive
lat = -76.449 # west-negative #west of england

#pos = pvlib.solarposition.get_solarposition(date_series,lat, lon)
#pos

import math

solar_constant_e = 1367.0
DTR = 0.017453292  # Pi / 180
RTD = 57.2957795  # 180 / Pi


time_zone = 5*15 # west-positive


def month_and_day_to_julian_day(month, day):
    if (month == 1): return day
    if (month == 2): return day + 31
    if (month == 3): return day + 59
    if (month == 4): return day + 90
    if (month == 5): return day + 120
    if (month == 6): return day + 151
    if (month == 7): return day + 181
    if (month == 8): return day + 212
    if (month == 9): return day + 243
    if (month == 10): return day + 273
    if (month == 11): return day + 304
    if (month == 12):
        return day + 334
    else:
        print("bad month")


def solar_elev_azi_ecc(latitude,
                       longitude,
                       time_zone,
                       jday,
                       time,
                       solar_time=0):
    # /*  angles in degrees, times in hours  */

    # /*  solar elevation and azimuth formulae from sun.c  */
    if (solar_time == 1): sol_time = time
    if (solar_time == 0):
        sol_time = time + 0.170 * math.sin(
            (4 * math.pi / 373) * (jday - 80)) - 0.129 * math.sin(
                (2 * math.pi / 355) *
                (jday - 8)) + 12 / 180.0 * (time_zone - longitude)

    solar_declination = RTD * 0.4093 * math.sin(
        (2 * math.pi / 368) * (jday - 81))
    jday_angle = 2 * math.pi * (jday - 1) / 365

    solar_elevation = RTD * math.asin(
        math.sin(latitude * DTR) * math.sin(solar_declination * DTR) -
        math.cos(latitude * DTR) * math.cos(solar_declination * DTR) *
        math.cos(sol_time * (math.pi / 12)))

    solar_azimuth = RTD * (-math.atan2(
        math.cos(solar_declination * DTR) * math.sin(sol_time *
                                                     (math.pi / 12)),
        -math.cos(latitude * DTR) * math.sin(solar_declination * DTR) -
        math.sin(latitude * DTR) * math.cos(solar_declination * DTR) *
        math.cos(sol_time * (math.pi / 12))))

    # /*  eccentricity_correction formula used in genjdaylit.c */

    eccentricity_correction = 1.00011 + 0.034221 * math.cos(
        jday_angle) + 0.00128 * math.sin(jday_angle) + 0.000719 * math.cos(
            2 * jday_angle) + 0.000077 * math.sin(2 * jday_angle)

    return solar_elevation, solar_azimuth, eccentricity_correction


def diffuse_fraction(irrad_glo, solar_elevation, eccentricity_correction):
    dif_frac = 0

    if (solar_elevation > 0):
        irrad_ex = solar_constant_e * eccentricity_correction * math.sin(
            DTR * solar_elevation)
    else:
        irrad_ex = 0

    if (irrad_ex > 0):
        index_glo_ex = irrad_glo / irrad_ex
    else:
        return 0

    if (index_glo_ex < 0): print("negative irrad_glo in diffuse_fraction_th\n")
    if (index_glo_ex > 1): index_glo_ex = 1

    if (index_glo_ex <= 0.3):
        dif_frac = 1.02 - 0.254 * index_glo_ex + 0.0123 * math.sin(
            DTR * solar_elevation)
    if (dif_frac > 1):
        dif_frac = 1

    if (0.3 < index_glo_ex < 0.78):
        dif_frac = 1.4 - 1.749 * index_glo_ex + 0.177 * math.sin(
            DTR * solar_elevation)
    if (dif_frac > 0.97):
        dif_frac = 0.97
    if (dif_frac < 0.1): dif_frac = 0.1

    if (index_glo_ex >= 0.78):
        dif_frac = 0.486 * index_glo_ex - 0.182 * math.sin(
            DTR * solar_elevation)
    if dif_frac < 0.1:
        dif_frac = 0.1

    return dif_frac


def calc_split(month, day, time, irrad_glo):
    jday = month_and_day_to_julian_day(month, day)

    if (irrad_glo < 0 or irrad_glo > solar_constant_e
        ):  # / * check irradiances and exit if necessary * /
        irrad_glo = solar_constant_e

    # no object orientation yet

    solar_elevation, solar_azimuth, eccentricity_correction = solar_elev_azi_ecc(
        lat, lon, time_zone, jday, time)

    #

    irrad_dif = diffuse_fraction(irrad_glo, solar_elevation,
                                 eccentricity_correction) * irrad_glo

    if (solar_elevation > 5.0):

        irrad_beam_nor = (irrad_glo - irrad_dif) * 1.0 / math.sin(
            DTR * solar_elevation)

    else:
        irrad_beam_nor = 0
        irrad_dif = irrad_glo

    if (irrad_beam_nor > solar_constant_e):
        irrad_beam_nor = solar_constant_e
        irrad_dif = irrad_glo - irrad_beam_nor * math.sin(
            DTR * solar_elevation)

    #print(month, day, round(time, 2), round(irrad_beam_nor, 2), round(irrad_dif, 2))

    return irrad_beam_nor, irrad_dif


#print("month, day, time, GHR\n")
calc_split(4, 22, 8.333333333, 107)



def calc_split_df(df):
    df['hour'] = df.index.hour
    df['day'] = df.index.day
    df['month'] = df.index.month
    df['DHR'] = 0
    df['DNI'] = 0
    
    i = 0    
    for index, row in df.iterrows():
        #while i <50:
            GHR = df.iloc[i,df.columns.get_loc('Solar Radiation')].values[0]
            DNI, DHR = calc_split(df.month[i], df.day[i], df.hour[i], GHR)
            print(i,  GHR, DNI, DHR)
            df.iloc[i, df.columns.get_loc('DHR')] = DHR
            df.iloc[i, df.columns.get_loc('DNI')] = DNI
            i+=1
            
    return df


df5 = calc_split_df(df5)

0 0.0 0.0 0.0
1 1.163 0.15101535087394616 1.12811
2 1.163 0.12610880732101337 1.12811
3 0.0 0.0 0.0
4 0.0 0.0 0.0
5 0.0 0.0 0.0
6 0.0 0.0 0.0
7 0.0 0.0 0.0
8 0.0 0.0 0.0
9 2.326 0.11844711821844454 2.25622
10 22.096999999999998 1.1143199613387635 21.434089999999998
11 30.238000000000003 1.5480700178418183 29.33086
12 38.379 2.0451623321369516 37.22763
13 17.445 0.992240867074542 16.92165
14 12.793 0.7965542395109909 12.409209999999998
15 12.793 0.8942134671180086 12.409209999999998
16 15.119000000000002 1.2154558742288846 14.66543
17 3.489 0.3296089331870705 3.38433
18 0.0 0.0 0.0
19 0.0 0.0 0.0
20 0.0 0.0 0.0
21 0.0 0.0 0.0
22 0.0 0.0 0.0
23 0.0 0.0 0.0
24 0.0 0.0 0.0
25 0.0 0.0 0.0
26 0.0 0.0 0.0
27 0.0 0.0 0.0
28 0.0 0.0 0.0
29 0.0 0.0 0.0
30 0.0 0.0 0.0
31 0.0 0.0 0.0
32 1.163 0.0614650142107219 1.12811
33 17.445 0.8903027580525292 16.92165
34 58.15 2.938244797790477 56.405499999999996
35 94.20299999999999 4.831724917203087 91.37690999999998
36 110.485 5.898005900422579 107.17045
3

254 227.94799999999998 58.07458017596133 200.76297010659624
255 198.873 56.684275541319074 175.3251745559572
256 168.635 55.041840957109784 148.8724099364888
257 26.749000000000002 2.6499462552498314 25.946530000000003
258 0.0 0.0 0.0
259 0.0 0.0 0.0
260 0.0 0.0 0.0
261 0.0 0.0 0.0
262 0.0 0.0 0.0
263 0.0 0.0 0.0
264 0.0 0.0 0.0
265 0.0 0.0 0.0
266 0.0 0.0 0.0
267 0.0 0.0 0.0
268 0.0 0.0 0.0
269 0.0 0.0 0.0
270 0.0 0.0 0.0
271 0.0 0.0 0.0
272 3.489 0.19085065918477512 3.38433
273 32.564 1.7146867012999347 31.58708
274 95.366 4.9612319673013765 92.50502
275 153.516 8.097425677136613 148.91052
276 211.666 16.585662019847756 202.60111365500345
277 232.6 32.61193658962773 215.9172389321953
278 204.688 26.553491626998234 192.30992421035504
279 145.375 13.382072075291761 139.84217478366756
280 65.128 5.472502718056682 63.17416
281 15.119000000000002 1.5082027618426728 14.66543
282 0.0 0.0 0.0
283 0.0 0.0 0.0
284 0.0 0.0 0.0
285 0.0 0.0 0.0
286 0.0 0.0 0.0
287 0.0 0.0 0.0
288 0.0 0.0 0.0
289 

550 0.0 0.0 0.0
551 0.0 0.0 0.0
552 0.0 0.0 0.0
553 0.0 0.0 0.0
554 0.0 0.0 0.0
555 0.0 0.0 0.0
556 0.0 0.0 0.0
557 0.0 0.0 0.0
558 0.0 0.0 0.0
559 0.0 0.0 0.0
560 1.163 0.06796972436938481 1.12811
561 10.467 0.5860785428237456 10.15299
562 30.238000000000003 1.668379067890058 29.33086
563 53.498000000000005 2.9909029208528572 51.893060000000006
564 40.705 2.371570211275236 39.48385
565 60.476000000000006 3.778818393982007 58.66172
566 44.193999999999996 3.0506863973661926 42.868179999999995
567 39.542 3.109861892905444 38.35574
568 25.586 2.36687226648734 24.818419999999996
569 12.793 1.4371838798608376 12.409209999999998
570 0.0 0.0 0.0
571 0.0 0.0 0.0
572 0.0 0.0 0.0
573 0.0 0.0 0.0
574 0.0 0.0 0.0
575 0.0 0.0 0.0
576 0.0 0.0 0.0
577 0.0 0.0 0.0
578 0.0 0.0 0.0
579 0.0 0.0 0.0
580 0.0 0.0 0.0
581 0.0 0.0 0.0
582 0.0 0.0 0.0
583 0.0 0.0 0.0
584 3.489 0.2053096836219542 3.38433
585 109.322 6.160546413670638 106.04234
586 236.08900000000003 19.9822270431565 225.29104140882282
587 274.4

831 324.47700000000003 698.5394877455949 87.73083304473191
832 202.362 320.81487389481185 112.23895474177856
833 70.943 11.070141576783367 68.47783459511143
834 3.489 0.6223507124057314 3.38433
835 0.0 0.0 0.0
836 0.0 0 0.0
837 0.0 0 0.0
838 0.0 0 0.0
839 0.0 0 0.0
840 0.0 0 0.0
841 0.0 0.0 0.0
842 0.0 0.0 0.0
843 0.0 0.0 0.0
844 0.0 0.0 0.0
845 0.0 0.0 0.0
846 0.0 0.0 0.0
847 0.0 0.0 0.0
848 10.467 0.6741873608027702 10.15299
849 97.69200000000001 5.9923225370080395 94.76124
850 143.049 8.623095317665628 138.75753
851 173.287 13.298859673483669 166.7581498675483
852 408.213 520.1471344525534 164.09267866652053
853 354.715 439.5653609696421 163.7973729488484
854 340.759 545.7704409860218 128.82095820509207
855 203.525 180.8452178981974 143.04116919579585
856 38.379 4.166010801691219 37.22763
857 13.956 1.9201048380834995 13.53732
858 1.163 0.21344643191148802 1.12811
859 0.0 0.0 0.0
860 0.0 0 0.0
861 0.0 0 0.0
862 0.0 0 0.0
863 0.0 0 0.0
864 0.0 0 0.0
865 0.0 0.0 0.0
866 0.0 0.0 0.0
86

1115 120.95200000000001 8.284030311376162 117.32344
1116 232.6 125.18934988072286 180.50478097319836
1117 264.001 276.2381482734472 158.85563966490298
1118 330.292 766.8520212964024 74.20736567606046
1119 370.99699999999996 783.9427110024658 152.08702859804646
1120 154.679 307.22928170750777 87.00880656798597
1121 147.701 657.1529236902778 41.89011231803895
1122 12.793 3.6363258959976092 12.409209999999998
1123 0.0 0 0.0
1124 0.0 0 0.0
1125 0.0 0 0.0
1126 0.0 0 0.0
1127 0.0 0 0.0
1128 0.0 0 0.0
1129 0.0 0 0.0
1130 0.0 0.0 0.0
1131 0.0 0.0 0.0
1132 0.0 0.0 0.0
1133 0.0 0.0 0.0
1134 0.0 0.0 0.0
1135 0.0 0.0 0.0
1136 19.771 1.457359905748294 19.17787
1137 100.01799999999999 6.965763457229209 97.01745999999999
1138 248.882 129.79052235933108 191.83466712755654
1139 427.98400000000004 750.0177959778389 103.43127806863612
1140 411.702 780.3579900960088 91.1327415834866
1141 434.962 772.1361168363436 145.22464458139294
1142 489.62300000000005 855.2638048844045 208.68541462760643
1143 177.9390

...

7362 0.0 0.0 0.0
7363 0.0 0 0.0
7364 0.0 0 0.0
7365 0.0 0 0.0
7366 0.0 0 0.0
7367 0.0 0 0.0
7368 0.0 0 0.0
7369 0.0 0.0 0.0
7370 0.0 0.0 0.0
7371 0.0 0.0 0.0
7372 0.0 0.0 0.0
7373 0.0 0.0 0.0
7374 0.0 0.0 0.0
7375 2.326 0.158526203981523 2.25622
7376 61.638999999999996 3.9313734553573214 59.789829999999995
7377 110.485 6.814014468763659 107.17045
7378 129.093 7.947584557960166 125.22020999999998
7379 112.811 7.156784891425124 109.42667
7380 181.428 20.073336107927005 172.51160095100744
7381 173.287 29.76962510032133 161.28483966123343
7382 137.234 18.197017399830973 130.8175453984222
7383 86.06200000000001 8.723581201810283 83.48014
7384 52.335 6.622234958474878 50.76495
7385 23.26 3.87676424950954 22.5622
7386 0.0 0.0 0.0
7387 0.0 0 0.0
7388 0.0 0 0.0
7389 0.0 0 0.0
7390 0.0 0 0.0
7391 0.0 0 0.0
7392 0.0 0.0 0.0
7393 0.0 0.0 0.0
7394 0.0 0.0 0.0
7395 0.0 0.0 0.0
7396 0.0 0.0 0.0
7397 0.0 0.0 0.0
7398 0.0 0.0 0.0
7399 0.0 0.0 0.0
7400 11.63 0.7347365991886899 11.2811
7401 25.586 1.5635

7648 22.096999999999998 2.312359127161791 21.434089999999998
7649 1.163 0.15140773655880405 1.12811
7650 0.0 0.0 0.0
7651 0.0 0.0 0.0
7652 0.0 0.0 0.0
7653 0.0 0.0 0.0
7654 0.0 0.0 0.0
7655 0.0 0.0 0.0
7656 0.0 0.0 0.0
7657 0.0 0.0 0.0
7658 0.0 0.0 0.0
7659 0.0 0.0 0.0
7660 0.0 0.0 0.0
7661 0.0 0.0 0.0
7662 0.0 0.0 0.0
7663 0.0 0.0 0.0
7664 38.379 2.221173290799261 37.22763
7665 160.494 9.009120375900235 155.67918
7666 286.098 92.5152323275179 236.5485020226738
7667 376.81199999999995 296.72696224590464 222.0053435518963
7668 418.68 485.7683901975869 178.8173827774882
7669 405.88699999999994 571.1878578192058 146.74897419295422
7670 347.73699999999997 520.021377612591 137.56760796168584
7671 244.23 290.3992391498862 143.00469274734334
7672 118.626 20.616138232840843 112.6324071865982
7673 11.63 1.4875358020226006 11.2811
7674 0.0 0.0 0.0
7675 0.0 0.0 0.0
7676 0.0 0.0 0.0
7677 0.0 0.0 0.0
7678 0.0 0.0 0.0
7679 0.0 0.0 0.0
7680 0.0 0.0 0.0
7681 0.0 0.0 0.0
7682 0.0 0.0 0.0
7683 0.0 0.0 0

7942 0.0 0.0 0.0
7943 0.0 0.0 0.0
7944 0.0 0.0 0.0
7945 0.0 0.0 0.0
7946 0.0 0.0 0.0
7947 0.0 0.0 0.0
7948 0.0 0.0 0.0
7949 0.0 0.0 0.0
7950 0.0 0.0 0.0
7951 0.0 0.0 0.0
7952 2.326 0.12626570347644533 2.25622
7953 15.119000000000002 0.7968629621905818 14.66543
7954 23.26 1.2219132179990566 22.5622
7955 45.357 2.437903003358691 43.996289999999995
7956 54.661 3.0864284324751194 53.02117
7957 80.247 4.889450940697224 77.83959
7958 97.69200000000001 6.600880427944795 94.76124
7959 39.542 3.04574232527796 38.35574
7960 25.586 2.3082234367964047 24.818419999999996
7961 4.652 0.5037251403335818 4.51244
7962 0.0 0.0 0.0
7963 0.0 0.0 0.0
7964 0.0 0.0 0.0
7965 0.0 0.0 0.0
7966 0.0 0.0 0.0
7967 0.0 0.0 0.0
7968 0.0 0.0 0.0
7969 0.0 0.0 0.0
7970 0.0 0.0 0.0
7971 0.0 0.0 0.0
7972 0.0 0.0 0.0
7973 0.0 0.0 0.0
7974 0.0 0.0 0.0
7975 0.0 0.0 0.0
7976 3.489 0.18864101041860276 3.38433
7977 25.586 1.343160917544683 24.818419999999996
7978 50.00899999999999 2.6163139527198505 48.50872999999999
7979 68.617

8231 0.0 0.0 0.0
8232 0.0 0.0 0.0
8233 0.0 0.0 0.0
8234 0.0 0.0 0.0
8235 0.0 0.0 0.0
8236 0.0 0.0 0.0
8237 0.1163 0.007754957267302685 0.112811
8238 0.0 0.0 0.0
8239 0.0 0.0 0.0
8240 1.6282 0.0852857418848881 1.5793540000000001
8241 14.6538 0.7448972034020306 14.214186
8242 40.9376 2.0705656294080192 39.709472000000005
8243 20.4688 1.0561115999147712 19.854736000000003
8244 8.6062 0.46447127097620583 8.348014
8245 14.0723 0.8147220219633005 13.650131
8246 70.5941 4.497187453699874 68.476277
8247 87.5739 6.295306303746943 84.946683
8248 33.1455 2.7540829190158322 32.151135
8249 2.326 0.22809727376931532 2.25622
8250 0.0 0.0 0.0
8251 0.0 0.0 0.0
8252 0.0 0.0 0.0
8253 0.0 0.0 0.0
8254 0.0 0.0 0.0
8255 0.0 0.0 0.0
8256 0.0 0.0 0.0
8257 0.0 0.0 0.0
8258 0.0 0.0 0.0
8259 0.0 0.0 0.0
8260 0.0 0.0 0.0
8261 0.0 0.0 0.0
8262 0.0 0.0 0.0
8263 0.0 0.0 0.0
8264 2.5586 0.13378005684957084 2.4818420000000003
8265 48.1482 2.4428896483315152 46.703754
8266 117.3467 5.923027620757611 113.82629899999999


8512 79.8981 6.399333336143848 77.50115699999999
8513 8.141 0.7645999946561972 7.89677
8514 0.0 0.0 0.0
8515 0.0 0.0 0.0
8516 0.0 0.0 0.0
8517 0.0 0.0 0.0
8518 0.0 0.0 0.0
8519 0.0 0.0 0.0
8520 0.0 0.0 0.0
8521 0.0 0.0 0.0
8522 0.0 0.0 0.0
8523 0.0 0.0 0.0
8524 0.0 0.0 0.0
8525 0.0 0.0 0.0
8526 0.0 0.0 0.0
8527 0.0 0.0 0.0
8528 6.8617 0.3558004328113982 6.655849
8529 68.1518 3.4236434232465904 66.10724599999999
8530 183.6377 9.15928225499768 178.128569
8531 230.9718 16.77077092722347 221.05780979262627
8532 268.1878 40.16483541076096 245.41876578423327
8533 367.0428 250.699486459807 234.08127799111134
8534 300.6355 175.54089621063943 215.66432523620833
8535 209.2237 60.19903123403025 183.27072604330309
8536 92.3422 7.384075988959435 89.571934
8537 10.3507 0.9703452092767435 10.040179
8538 0.0 0.0 0.0
8539 0.0 0.0 0.0
8540 0.0 0.0 0.0
8541 0.0 0.0 0.0
8542 0.0 0.0 0.0
8543 0.0 0.0 0.0
8544 0.0 0.0 0.0
8545 0.0 0.0 0.0
8546 0.0 0.0 0.0
8547 0.0 0.0 0.0
8548 0.0 0.0 0.0
8549 0.0 0.0 0.0
8

# Variables in the epw that we need


# From GFR
average temperature (F),
 relative humidity (%), 
solar radiation (langleys), 
10 meter wind speed (mph), 
wind direction (degrees), 
precipitation (inches)

# To EPW

average temperature (°C),
relative humidity (%), 
GHI  (Wh/m2), 
DNI  (Wh/m2), 
DHI  (Wh/m2), 
10 meter wind speed (m/s), 
wind direction (degrees), 
precipitation (cm or mm)


# Make the 2019 file
# Make the 2020 file and replace half of the year with the values from 2019

 df5_2020[start_time:end_time] = df5_2019.loc[start_time:end_time]



In [ ]:
display (df5)

In [13]:
import pandas as pd
from pandas import read_csv
import csv
import unittest
import numpy as np
from pprint import pprint
from setuptools import setup, find_packages
from codecs import open
from os import path
from epw import epw

#S5 = HoboReader("DL5_Game_Farm_Road_2019.csv")
#df5 = S5.get_dataframe()


## Template

template=epw()
template.read('USA_NY_Ithaca_Template.epw')
template.dataframe.columns


## Real file
    
ithaca_2019=epw()
#ithaca_2019.headers['LOCATION'][0]='Ithaca_NY'

ithaca_2019.headers = {'LOCATION': ['Ithaca',
  'NY',
  'USA',
  'TMY3',
  '725190',
  '42.438934',
  '-76.492435',
  '-5.0',
  '125.0'],
 'DESIGN CONDITIONS': ['0'],
 'TYPICAL/EXTREME PERIODS': ['0'],
 'GROUND TEMPERATURES': ['3',
  '0.5',
  '',
  '',
  '',
  '2.44',
  '4.62',
  '9.34',
  '15.56',
  '21.51',
  '25.58',
  '26.67',
  '24.44',
  '19.55',
  '13.31',
  '7.40',
  '3.42',
  '2',
  '',
  '',
  '',
  '5.55',
  '6.08',
  '8.78',
  '13.07',
  '17.76',
  '21.58',
  '23.50',
  '23.98',
  '20.18',
  '15.85',
  '11.17',
  '7.40',
  '4',
  '',
  '',
  '',
  '8.79',
  '8.41',
  '9.60',
  '12.12',
  '15.30',
  '18.27',
  '20.23',
  '20.66',
  '19.42',
  '16.85',
  '13.67',
  '10.73'],
 'HOLIDAYS/DAYLIGHT SAVINGS': ['No', '0', '0', '0'],
 'COMMENTS 1': [''],
 'COMMENTS 2': [''],
 'DATA PERIODS': ['1', '1', 'Data', 'Sunday', '1/1', '12/31']}

print(ithaca_2019.headers )


#variables = ["Wind Speed","MRT","Temp","RH","Solar Radiation","Wind Speed","Gust Speed","Wind Direction","DewPoint" ]

ithaca_2019.dataframe['Year'] = np.array(np.ones(8760)*2019).astype(int)
ithaca_2019.dataframe['Month'] = template.dataframe['Month'].astype(int)
ithaca_2019.dataframe['Day']  = template.dataframe['Day'].astype(int)
ithaca_2019.dataframe['Hour'] = template.dataframe['Hour'].astype(int)
ithaca_2019.dataframe['Minute'] = template.dataframe['Minute'].astype(int)     
ithaca_2019.dataframe['Data Source and Uncertainty Flags'] =  np.array(np.zeros(8760)).astype(int)



## Actual file starts here
#6
ithaca_2019.dataframe['Dry Bulb Temperature']=df5['Temp'].values.flatten() #.apply(lambda x: ftoc(x), axis=1).values.flatten()

#     Add this later (ADDED TO DL5_2019 FILE 07.22.2020) 
ithaca_2019.dataframe['Dew Point Temperature'] =df5['DewPoint'].values.flatten()  #np.array(np.zeros(8760)).astype(int)


ithaca_2019.dataframe['Relative Humidity'] =df5['RH'].values.flatten()
#     
ithaca_2019.dataframe['Atmospheric Station Pressure'] =  np.array(np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Extraterrestrial Horizontal Radiation'] =  np.array(np.zeros(8760)).astype(int)
#     
ithaca_2019.dataframe['Extraterrestrial Direct Normal Radiation'] =  np.array(np.zeros(8760)).astype(int)
#     
ithaca_2019.dataframe['Horizontal Infrared Radiation Intensity']=  np.array(np.zeros(8760)).astype(int)
#     
ithaca_2019.dataframe['Global Horizontal Radiation'] =  df5['Solar Radiation'].values.flatten()
ithaca_2019.dataframe['Direct Normal Radiation'] = df5['DNI'].values.flatten()    
ithaca_2019.dataframe['Diffuse Horizontal Radiation']= df5['DHR'].values.flatten()

# Do we need this?
ithaca_2019.dataframe['Global Horizontal Illuminance'] =  np.array(np.zeros(8760)).astype(int)   
ithaca_2019.dataframe['Direct Normal Illuminance']=  np.array(np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Diffuse Horizontal Illuminance']    =  np.array(np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Zenith Luminance']=  np.array(np.zeros(8760)).astype(int)

ithaca_2019.dataframe['Wind Direction'] = df5['Wind Direction'].values.flatten().astype(int)
ithaca_2019.dataframe['Wind Speed'] = df5['Wind Speed'].values.flatten()


ithaca_2019.dataframe['Total Sky Cover']  =  np.array(np.zeros(8760)).astype(int)
#     This is from Ithaca airport data (NWS, NOAA) (ADDED TO DL5_2019 FILE 07.22.2020) 
ithaca_2019.dataframe['Opaque Sky Cover (used if Horizontal IR Intensity missing)']= df5['SkyCover']
#   

ithaca_2019.dataframe['Visibility'] =  np.array(np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Ceiling Height'] =  np.array(np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Present Weather Observation'] =  np.array(np.zeros(8760)).astype(int)
#     
ithaca_2019.dataframe['Present Weather Codes'] =  np.array(np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Precipitable Water'] =  np.array(np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Aerosol Optical Depth'] =  np.array(np.zeros(8760)).astype(int)
#     
ithaca_2019.dataframe['Snow Depth'] =  np.array(np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Days Since Last Snowfall']=  np.array(np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Albedo'] =  np.array(np.zeros(8760)).astype(int)
#     

# We have precipitation but lets not use it for now
ithaca_2019.dataframe['Liquid Precipitation Depth']=  np.array(np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Liquid Precipitation Quantity']=  np.array(np.zeros(8760)).astype(int)




print(ithaca_2019.dataframe)





ithaca_2019.write('Ithaca_2019.epw')







{'LOCATION': ['Ithaca', 'NY', 'USA', 'TMY3', '725190', '42.438934', '-76.492435', '-5.0', '125.0'], 'DESIGN CONDITIONS': ['0'], 'TYPICAL/EXTREME PERIODS': ['0'], 'GROUND TEMPERATURES': ['3', '0.5', '', '', '', '2.44', '4.62', '9.34', '15.56', '21.51', '25.58', '26.67', '24.44', '19.55', '13.31', '7.40', '3.42', '2', '', '', '', '5.55', '6.08', '8.78', '13.07', '17.76', '21.58', '23.50', '23.98', '20.18', '15.85', '11.17', '7.40', '4', '', '', '', '8.79', '8.41', '9.60', '12.12', '15.30', '18.27', '20.23', '20.66', '19.42', '16.85', '13.67', '10.73'], 'HOLIDAYS/DAYLIGHT SAVINGS': ['No', '0', '0', '0'], 'COMMENTS 1': [''], 'COMMENTS 2': [''], 'DATA PERIODS': ['1', '1', 'Data', 'Sunday', '1/1', '12/31']}
      Year  Month  Day  Hour  Minute  Data Source and Uncertainty Flags  \
0     2019      1    1     1       0                                  0   
1     2019      1    1     2       0                                  0   
2     2019      1    1     3       0                            

In [ ]:
a=epw()
a.read(r"C:\Users\remym\Dropbox\Cornell 2019-2020\Grasshopper\Weather Files\MorphedWeather\USA_NY_Ithaca_HadCM3-A2-2080.epw")
a.dataframe
